# Reorder rendered images and read datasets metrics
```
results/
    -dataset_i/  
        - scene_j/
            - results.json
            - per_view.json
            - compact3dgs
                - img_0
                 ...
                - img_n
            - gs
            - featsplat_16
            - featsplat_32
            - semfeatsplat_32
```

# This file is under the MIT License and not bounded by Gaussian Splatting License
#MIT License
#
#Copyright (c) 2024 Tomás Berriel Martins
#
#Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell  copies of the Software, and to permit persons to whom Christopher Johannes Wewerthe Software is furnished to do so, subject to the following conditions:
# 
#The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.
#
#THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.


In [6]:
#  imports and define base directory

from pathlib import Path
import numpy as np
import shutil
import os
import json

base_dir = "/home/tberriel/Workspaces/splatting_ws/featsplat/data/output/"


In [3]:
# Move images generated by the models trained to fit the defined hierarchy

models = ["gs","c3dgs","featsplat_16","featsplat_32","semfeatsplat_32"]
for model in models:
    model_folder = os.path.join(base_dir,"eval", model)
    datasets = os.listdir(model_folder)
    for dataset in datasets:
        dataset_folder = os.path.join(model_folder, dataset)
        if os.path.isdir(dataset_folder):
            scenes = os.listdir(dataset_folder)
            for scene in scenes:
                # Source folder path
                source_folder = os.path.join(dataset_folder,scene, "test/ours_30000/renders/")
                # Destination folder path
                os.makedirs(os.path.join(base_dir,"images",dataset,scene),exist_ok=True)
                destination_folder = os.path.join(base_dir,"images",dataset,scene,model)

                # Copy the folder to the new path
                shutil.copytree(source_folder, destination_folder)#, dirs_exist_ok=True)

# Move gt            
datasets = os.listdir(os.path.join(base_dir,"eval/featsplat_32"))
for dataset in datasets:
    dataset_folder = os.path.join(model_folder, dataset)
    if os.path.isdir(dataset_folder):
        scenes = os.listdir(dataset_folder)
        for scene in scenes:
            # Source folder path
            source_folder = os.path.join(dataset_folder,scene, "test/ours_30000/gt/")
            # Destination folder path
            os.makedirs(os.path.join(base_dir,"images",dataset,scene),exist_ok=True)
            destination_folder = os.path.join(base_dir,"images",dataset,scene,"gt")

            # Copy the folder to the new path
            shutil.copytree(source_folder, destination_folder)#, dirs_exist_ok=True)


In [4]:
# copy results.json and per_view.json info from trained models to the global file
datasets = ["360_v2","db","tandt","scannetpp"]
for dataset in datasets:
    dataset_folder = os.path.join(base_dir,"images", dataset)
    if os.path.isdir(dataset_folder):
        scenes = os.listdir(dataset_folder)
        for scene in scenes:
            scene_folder = os.path.join(dataset_folder, scene)
            scene_results ={}
            scene_per_view = {}
            # Source folder path
            for model in models:
                model_folder = os.path.join(base_dir,"eval", model, dataset, scene)
                if os.path.exists(model_folder+"/results.json"):
                    with open(model_folder+"/results.json", 'r') as fp:
                        model_results = json.load(fp)
                        scene_results[model] = model_results["ours_30000"]
                    with open(model_folder+"/per_view.json", 'r') as fp:
                        model_per_view = json.load(fp)
                        scene_per_view[model] = model_per_view["ours_30000"]

            with open(scene_folder + "/results.json", 'w') as fp:
                json.dump(scene_results, fp, indent=True)
            with open(scene_folder + "/per_view.json", 'w') as fp:
                json.dump(scene_per_view, fp, indent=True)

In [7]:
# Read metrics from defined paths/downloaded images
base_path = Path(base_dir+"/results")
for dataset in base_path.iterdir():
    dataset_results = dict()
    dataset_per_view_results = dict()
    dataset_folder = base_path / dataset
    for scene in dataset_folder.iterdir():
        scene_folder = dataset_folder / scene
        with open(scene_folder / "results.json", 'r') as fp:
            scene_results = json.load(fp)
            for model, items in scene_results.items():
                if model not in list(dataset_results.keys()):
                    dataset_results[model] = dict()
                for metric, values in items.items():
                    if metric not in list(dataset_results[model].keys()):
                        dataset_results[model][metric] = []
                    dataset_results[model][metric].extend([values])
                    
    print(f"\n Dataset {dataset.name} \n")
    padding = max([len(x) for x in dataset_results.keys()])+1
    for model, model_items in dataset_results.items():
        metrics_str = ""
        for key, metric in model_items.items():
            metrics_str += " {} {:.3f} ".format(key, np.array(metric).mean())
        print(" {} : {};".format(model.ljust(padding), metrics_str))



 Dataset scannetpp 

 c3dgs            :  SSIM 0.854  PSNR 22.756  LPIPS 0.272  fps 170.144  size 35.397  n_gaussians 598226.286 ;
 featsplat_32     :  SSIM 0.875  PSNR 24.617  LPIPS 0.245  fps 70.836  size 112.342  n_gaussians 640141.333 ;
 featsplat_16     :  SSIM 0.875  PSNR 24.621  LPIPS 0.245  fps 81.924  size 73.607  n_gaussians 643116.143 ;
 gs               :  SSIM 0.867  PSNR 23.831  LPIPS 0.249  fps 184.486  size 217.104  n_gaussians 917939.381 ;
 semfeatsplat_32  :  SSIM 0.875  PSNR 24.639  LPIPS 0.244  fps 56.258  size 112.622  n_gaussians 641645.048 ;

 Dataset db 

 c3dgs         :  SSIM 0.901  PSNR 29.720  LPIPS 0.258  fps 184.782  size 43.005  n_gaussians 1045739.000 ;
 featsplat_32  :  SSIM 0.904  PSNR 29.848  LPIPS 0.247  fps 97.996  size 327.169  n_gaussians 1864393.000 ;
 featsplat_16  :  SSIM 0.904  PSNR 29.777  LPIPS 0.249  fps 111.265  size 211.636  n_gaussians 1849228.500 ;
 gs            :  SSIM 0.899  PSNR 29.464  LPIPS 0.247  fps 125.667  size 662.117  n_gau